### Project: Analyze fluctuations in HTTP request latency data to determine reliability

- Load data (.csv) to dataframe
- Standardize and cleaning data
- Analyze data:
    ABC
- Apply model

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [10]:
# Load dataset from CSV file
root_df = pd.read_csv('../statics/csv/raw_data.csv')

Dataset là những dữ liệu thời gian thực (time-series data) liên quan đến thông tin độ trễ (latency) từ người dùng truy cập dịch vụ.

**Details:**

*Dữ liệu là các metrics được trích xuất từ Prometheus. Metrics này liên quan đến thông tin request từ client được gửi tới Proxy. Các Exporter collect metrics từ proxy và đẩy nó về Prometheus database.* 

### Standardize and cleaning data

In [11]:
# Clean data
# Select related columns
cols = root_df.columns.to_list()
colums_drop = []
for col in cols:
    num_row = root_df.drop_duplicates(subset=col).__len__()
    if num_row == 1:
        colums_drop.append(col)
# Show columns will be drop
print("Columns will be drop: ", colums_drop)
root_df = root_df.drop(columns=colums_drop) # Only select column selected
# Rename columns
cols_rename_dict = {
    "Time": "timestamp",
    "Value": "value"
}
root_df = root_df.rename(columns=cols_rename_dict)
# Change datetime format for column "timestamp"
root_df['timestamp'] = pd.to_datetime(root_df["timestamp"], unit="ms")
root_df.head()

Columns will be drop:  ['__name__', 'app', 'controller_class', 'controller_namespace', 'controller_pod', 'host', 'ingress', 'instance', 'job', 'kubernetes_namespace', 'kubernetes_node', 'kubernetes_pod_name', 'method', 'namespace', 'ns', 'path', 'pod_template_hash', 'service']


,timestamp,le,status,value
0,2024-02-06 17:00:00,inf,200,109909316
1,2024-02-06 17:05:00,inf,200,109911193
2,2024-02-06 17:00:00,inf,206,10882
3,2024-02-06 17:05:00,inf,206,10882
4,2024-02-06 17:00:00,inf,304,10409248


Trên đây dataset liên quan đến thống kê truy cập từ client sau khi thông qua các bước clean và standardilize. Thông tin các cột xuất hiện trong bảng dữ liệu:
- timestamp: Dấu thời gian, biểu thị số liệu tại thời điểm đó
- le: Latency, các buckets latency (hay còn gọi là các bins theo cách gọi của histogram).
- status: HTTP status return by service
- value: Giá trị của latency tại thời điểm timestamp, nó sẽ luôn luôn tăng (counter) vì toàn bộ số lượng request gửi tới server từ trước tới nay luôn luôn tăng chứ không giảm.

*Note*

- Dữ liệu trên được scrape từ Proxy sau mỗi 5 minutes (interval=5m) và được aggregate trong thời gian đó, vì vậy timestamp luôn luôn có delta_t = 5 minutes.
- Việc lưu thông tin về request liên quan đến latency sử dụng các buckets để thống kê giúp cho việc giảm thiểu tài nguyên lưu trữ thay vì lưu lại duration_time cho mỗi request cụ thể. Vì vậy nó giúp cho việc overview tình trạng latency service hiệu quả hơn (dễ dàng overview bằng histogram). Tuy nhiên, sẽ phải đánh đổi về độ chính xác của metrics, đặc biệt là khi có những request có latency cực cao hoặc cực thấp thì histogram sẽ không phản ánh rõ được điều đó. 